In [82]:
import cv2
import time
import numpy as np
import math
import HandTrackingModule as htm
import os

In [51]:
wCam, hCam = 640, 480

In [64]:
detector = htm.handDetector(detectionCon=0.7)


In [118]:
cap = cv2.VideoCapture(0)
cap.set(5, wCam)
cap.set(5, hCam)
pTime = 0

volRange = [0, 100]
minVol = volRange[0]
maxVol = volRange[1]

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        #print(lmList)
        #print(lmList[4], lmList[8])

        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

        length = math.hypot(x2 - x1, y2 - y1)
        #print(length)

        vol = np.interp(length, [50, 300], [minVol, maxVol])
        print(int(length), vol)


        def set_volume(volume):
            os.system(f"osascript -e 'set volume output volume {volume}'")
        set_volume(100)

        vol = np.interp(length, [50, 300], [minVol, maxVol])
        volBar = np.interp(length, [50, 300], [400, 150])
        volPer = np.interp(length, [50, 300], [0, 100])
        print(int(length), vol)

        if length < 50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX, 
                1, (255, 0, 0), 3)
    cv2.imshow("Img", img)
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

/Users/my/myenv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


311 100.0
311 100.0
250 80.06797689570826
250 80.06797689570826
252 80.9015361627364
252 80.9015361627364
262 85.06074433393283
262 85.06074433393283
268 87.36479870050519
268 87.36479870050519
269 87.77457956308622
269 87.77457956308622
274 89.8457099754014
274 89.8457099754014
270 88.31805020401725
270 88.31805020401725
268 87.46534325074295
268 87.46534325074295
273 89.44880081572387
273 89.44880081572387
269 87.61858575543539
269 87.61858575543539
267 87.11713214981067
267 87.11713214981067
262 85.13381948735622
262 85.13381948735622
260 84.39386955180846
260 84.39386955180846
264 85.60303025955268
264 85.60303025955268
266 86.49694831308548
266 86.49694831308548
266 86.49694831308548
266 86.49694831308548
261 84.7091209016674
261 84.7091209016674
260 84.35784589574472
260 84.35784589574472
247 78.96504433384547
247 78.96504433384547
241 76.47300140453805
241 76.47300140453805
246 78.42357441182475
246 78.42357441182475
247 79.12981388058792
247 79.12981388058792
243 77.50897394599

In [ ]:
def set_volume(volume):
    os.system(f"osascript -e 'set volume output volume {volume}'")
set_volume(100)